# GPT from scratch

このファイルではGPTモデルの実装を目標とする。このファイルの立ち位置は同リポジトリに含まれている<br>
・pytorch_command.ipynb<br>
・attention_from_scratch.ipynb<br>
の次に読むことを想定されている。pytorchの下位APIは上２つの.ipynbで散々練習したので、上位APIを解禁して最新モデルを組むことを目指す<br>

pytorchに明るい人はこのipynbファイルから読んでも良い、pytorchに明るくない人は上２つのipynbファイルを公式ドキュメントとにらめっこして読むことをおすすめする。<br>
内容がわかりにくいと感じた人はGithub上もしくはXで作者に質問/書き換え要請を投げることができる。<br>

##### 本題
実装においては[GPT from scratch](https://jaketae.github.io/study/gpt/)やKarpathyの[minGPT](https://github.com/karpathy/minGPT)を参考にする。<br>
実装に関してはこのipynbを見なくとも紹介した2つのサイトを見れば良い。<br>

実装に入る前に簡単なGPTの実装方針についての概要について解説する。今回実装するのは初代GPTである。<br>
初代GPTは以下のような構造をしている<br>
- GPTはTransformerのDecoderのみを用いたモデルである。
- Embedding_dimは768, MultiheadAttentionのHead数は12, TransformerDecoderブロック数は12である。
- FFN層の活性化関数はGELUである。

訓練時の注意事項も記載しておく
- OptimizerはAdam, 学習率の最大値は2.5e-4, 2000iteratonで最大値に達し、<br>その後はCOSINEスケジューラーによってスケジューリングを行う。attention_from_scratch.ipynbで紹介したwarm_upの改善版のようなものである。<br>

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
import warnings
warnings.simplefilter('ignore')
print("CUDA環境が壊れていないことを祈りながら確認->", torch.cuda.is_available())

CUDA環境が壊れていないことを祈りながら確認-> True


まずは学習につかうTransformerDecoderLayerを定義する。デフォルトにtorch.nn.TransformerDecoderLayerがあるが、<br>
カスタムレイヤーは製作したほうが早い
下のコードはattention_from_scratch.ipynbのTransformerDecoderLayerのFFN層の活性化関数をGELUに変えただけである。<br>

In [2]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, embedding_dim, ffn_dim, num_heads, drop_out_rate = 0., layer_eps=1e-05, batch_first = False):
        super().__init__()
        self.multiheadselfattention = nn.MultiheadAttention(embedding_dim, num_heads,batch_first=batch_first)
        self.dropout_selfattn = nn.Dropout(p = drop_out_rate)
        self.layernorm_selfattn = nn.LayerNorm(embedding_dim, eps = layer_eps)

        self.multiheadattention = nn.MultiheadAttention(embedding_dim, num_heads,batch_first=batch_first) 
        self.dropout_attn = nn.Dropout(p = drop_out_rate)
        self.layernorm_attn = nn.LayerNorm(embedding_dim, eps = layer_eps)

        self.ffn = nn.Sequential(nn.Linear(embedding_dim, ffn_dim), nn.GELU(), nn.Linear(ffn_dim, embedding_dim))#GELUに変更
        self.layernorm_ffn = nn.LayerNorm(embedding_dim, eps = layer_eps)
        self.dropout_ffn = nn.Dropout(p = drop_out_rate)
    def forward(self, src, tgt, pad_mask_self = None, mask_self=None, pad_mask = None, mask = None):
        dtgt, _ = self.multiheadselfattention(tgt,tgt,tgt,key_padding_mask = pad_mask_self, attn_mask = mask_self)
        dtgt = self.dropout_selfattn(dtgt)
        tgt = self.layernorm_selfattn(tgt+dtgt)
        dtgt, _ = self.multiheadattention(tgt, src, src, key_padding_mask = pad_mask, attn_mask = mask)
        dtgt = self.dropout_attn(dtgt)
        tgt = self.layernorm_attn(tgt+dtgt)
        dtgt = self.dropout_ffn(self.ffn(tgt))
        tgt = self.layernorm_ffn(dtgt + tgt)
        return tgt
    
class positional_encoding(nn.Module):
    def __init__(self, embedding_dim: int,T = 10000):
        super().__init__()
        #PE (1, max_sequence_length, embedding_dim)
        self.pe = torch.zeros(size=(1, T, embedding_dim))
        t = torch.arange(start = 1, end = T+1).reshape(T, 1)
        k_odd = torch.arange(start = 1, end = embedding_dim+1, step = 2)
        k_odd = k_odd.reshape(1, k_odd.size(0))
        k_even = torch.arange(start = 2, end = embedding_dim+1, step = 2)
        k_even = k_even.reshape(1,k_even.size(0))
        phase_odd = t / T**((k_odd//2 * 2) / embedding_dim) #<- ブロードキャスト機能を用いて効率よく計算
        phase_even = t / T**((k_even//2 * 2) / embedding_dim)
        self.pe[0,:,0::2] = torch.sin(phase_odd)
        self.pe[0,:,1::2] = torch.cos(phase_even)
        self.register_buffer("positional_encoding_weight", self.pe)
        self.embedding_dim = embedding_dim
    def forward(self, x: torch.Tensor)->torch.Tensor:
        """入力のxのサイズx: (batch_size, sequence_length, embedding_dim)"""
        #np.sqrt(embedding_dim)をかけることでベクトルのスケールを合わせている
        return np.sqrt(self.embedding_dim)*x + self.pe[:,:x.size(1),:].to(x.device) 

In [3]:
class TransformerDecoder(nn.Module):
    def __init__(self, tgt_vocab_size, embedding_dim, ffn_dim, num_heads, drop_out_rate = 0.,\
                  layer_eps=1e-05, batch_first = False, T = 10000, N = 1):
        super().__init__()
        self.embedding = nn.Embedding(tgt_vocab_size, embedding_dim,)
        self.positional_encoding = positional_encoding(embedding_dim, T)
        self.decoder = nn.ModuleList([TransformerDecoderLayer(embedding_dim, ffn_dim, num_heads, drop_out_rate,\
                                                               layer_eps, batch_first) for _ in range(N)])
    def forward(self, src, tgt, pad_mask_self = None, mask_self=None, pad_mask = None, mask = None):
        tgt = self.embedding(tgt)
        tgt = self.positional_encoding(tgt)
        for layer in self.decoder:
            tgt = layer(src, tgt, pad_mask_self = pad_mask_self,mask_self = mask_self, pad_mask = pad_mask, mask = mask)
        return tgt